# getting started

In this lab we will compare Chain of Though (Cot) and ReAct (Reason Act) Prompting

In [1]:
import openai
import re
import httpx
import os
import rich
import json
from openai import OpenAI


api_key = "placeholder"  
model = "mistral-small:latest"              # Other options here include "qwen3:32b"
base_url = "http://localhost:11434/v1/"

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)


print("Imports complete, Client initialized")


Imports complete, Client initialized


# Chain of Thought

A Chain of Thought (CoT) prompt is a prompting technique used with large language models (LLMs) where you explicitly guide the model to reason step-by-step before arriving at the final answer.

Instead of asking the model to directly give you an answer, a CoT prompt encourages it to "think aloud"—breaking down the problem, analyzing it in stages, and only then giving a conclusion. This helps improve accuracy, especially for complex reasoning, math, logic, or multi-hop questions.

Can be combined with few-shot prompting (i.e., giving multiple CoT examples before the actual question)

TODO: Need to explain, as a call out, few shor learning and prompting

In [2]:
# Static example to demonstrate format (few-shot learning)
prompt = """
You are a thoughtful and logical assistant. For every question, you will:
- Think step-by-step under a “Thought” section.
- Then write the final result under “Answer”.
- Always follow the structure shown below.

Use this format:
Question: <the question>
Thought: <your detailed reasoning>
Answer: <final answer>

Here are some examples:

Question: If a train leaves at 2 PM and takes 3 hours to reach its destination, what time does it arrive?
Thought: The train departs at 2 PM. If it travels for 3 hours, it will arrive at 2 + 3 = 5 PM.
Answer: 5 PM

Question: What is the capital of the country whose official language is French and borders Germany?
Thought: France is a country that borders Germany and has French as its official language. The capital of France is Paris.
Answer: Paris

Question: What is the sum of the first three even numbers?
Thought: The first three even numbers are 2, 4, and 6. Their sum is 2 + 4 + 6 = 12.
Answer: 12

Now answer the next question using the same format:
"""

question = "If Tom has 5 cookies and eats 2, how many does he have left?"

messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": question}
    ]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
)

print(completion.choices[0].message.content)


Question: If Tom has 5 cookies and eats 2, how many does he have left?

Tom starts with 5 cookies.

After eating 2 of them, we subtract the amount eated from initial
Answer :3


# ReAct Style

TODO Callout re React v ReACT and maybe 

1. Encourages explicit reasoning, not just end answers
1. Allows the model to interleave thoughts and actions
1. Great for use with tools or plugins (e.g., search, code exec, database)
1. Makes model behavior transparent and verifiable

## Template

1. Question: [user question]
1. Thought: [model's internal reasoning]
1. Action: [some action like Search(), Calculator(), API call]
1. Observation: [result of the action]

...repeat Thought → Action → Observation...

Answer: [final response to the question]

TODO: Explain/introduce "few shot prompting"

In [3]:
# Static example to demonstrate format (few-shot learning)

few_shot_example = (
    "Question: What is the capital of the country that borders Germany and has Vienna as its capital?\n"
    "Thought: I need to find which country has Vienna as its capital.\n"
    "Action:  Lookup('country with capital Vienna')\n"
    "Observation: Austria\n"
    "Thought: Now check if Austria borders Germany.\n"
    "Action:  Lookup('Does Austria border Germany?')\n"
    "Observation: Yes\n"    
    "Answer: The capital of Austria, which borders Germany, is Vienna."
    )

question = "what is elevation range for the area that the eastern sector of colorado orogeny extends into?"


Note - 
- In the last example for CoT, the system prompt got the examples.
- In this example, we keep pass the example data with each questions.
It is just 2 different styles to play with.


Because the LLM is thinking ie  Thought → Action → Observation → Answer this will take a few moments 
<insert graphic of the `[*]` to the top left of the cell.  

In [4]:
# TODO: Talk about how prompt is an example of prompt engineering and assembling prompts on the fly


prompt = f"""{few_shot_example}

Question: {question}
Thought:"""

messages=[
        {"role": "system", "content": "Answer questions using a ReAct format: Thought → Action → Observation → Answer."},
        {"role": "user", "content": prompt}
    ]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
)

print(completion.choices[0].message.content)

I need to find more information about the Colorado Orogeny, its eastern boundary and the region it covers. There are two separate research areas here:
1. The Colorado Orogeny
2. Eastern Extension

Action:
  Lookup('Colorado Orogeny')
  Lookup('eastern sector of colorado orogeny')
Observation: The Colorado Orogeny is a geological event that formed the Rocky Mountains and was active around 70-45 million years ago. It affected multiple states in the western United States, The eastern extent covers parts of Wyoming, Montana North Dakota and South Dakota.

Thought: Now I need to find elevation details of a region covering these four states.

Action: Lookup('lowest highest point north dakota wyoming montana south dakota')
Observation:
Elevation range for areas in these 4 states are as follows (units are meters with feet equivalent where provided)
- Wyoming: Highest = 13,802(fm)4206,Lowest= 920(3015f)
-Montana: Highest = 12,799(3904f), Lowest= 1,800 (5900f))
 - South Dakota = Range 3,090.5 to

# Chain of Thought (CoT) vs ReAct (Reasoning + Acting)

## Chain of Thought (CoT)
What it is:
Just reasoning — step-by-step thoughts leading to an answer.


|Strengths|Limitations|
|:---|:---|
|Good for pure reasoning tasks (math, logic, factual multi-step questions).|Doesn’t interact with external tools or sources.|
|Easy to implement.|Limited when answers need fresh data, search, or database queries.|
|Transparent: you can see the reasoning path.||


## ReAct (Reasoning + Acting)
What it is:
A prompt style that combines reasoning (like CoT) with actions, such as calling tools, web searches, or internal functions. Often used with agents.


|Strengths|Limitations|
|---|---|
|Perfect for agent workflows, e.g., answering based on tool output, RAG systems, browsing, calling APIs.|More complex to implement (you need tool handlers or agents).|
|Enables decision-making with dynamic data.|Harder to debug if the chain gets too long or recursive.|
|You can plug in your own tools, like databases, vector search, etc.||





In [ ]:
## Summary

TODO: Create a summary of what we just saw
